# Transformer

In [1]:
from tensorflow import keras

filename = keras.utils.get_file(
    origin=(
        "https://storage.googleapis.com/download.tensorflow.org/"
        "data/shakespeare.txt"
    ),
)
shakespeare = open(filename, "r").read()

In [2]:
shakespeare[:250]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n'

In [3]:
import tensorflow as tf

sequence_length = 100

def split_input(input, sequence_length):
    for i in range(0, len(input), sequence_length):
        yield input[i : i + sequence_length]

features = list(split_input(shakespeare[:-1], sequence_length))
labels = list(split_input(shakespeare[1:], sequence_length))
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [7]:
x, y = next(dataset.as_numpy_iterator())

In [8]:
x[:50], y[:50]

(b'First Citizen:\nBefore we proceed any further, hear',
 b'irst Citizen:\nBefore we proceed any further, hear ')

In [14]:
from keras import layers

tokenizer = layers.TextVectorization(
    standardize=None,
    split=lambda x: tf.strings.unicode_split(x, 'UTF-8'),
    output_sequence_length=sequence_length,
)

tokenizer.adapt(dataset.map(lambda text, labels: text))

In [15]:
vocabulary_size = tokenizer.vocabulary_size()
vocabulary_size

67

In [16]:
dataset = dataset.map(
    lambda features, labels: (tokenizer(features), tokenizer(labels)),
    num_parallel_calls=8
)
trainning_dataset = dataset.shuffle(10_000).batch(64).cache()

In [18]:
embedding_dim = 256
hidden_dim = 1024

inputs = layers.Input(shape=(sequence_length,), dtype=tf.int32, name="token_ids")
x = layers.Embedding(vocabulary_size, embedding_dim)(inputs)
x = layers.GRU(hidden_dim, return_sequences=True)(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(vocabulary_size, activation="softmax")(x)
model = keras.Model(inputs, outputs, name="shakespeare")

In [19]:
model.summary(line_length=80)

Model: "shakespeare"
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
token_ids (InputLayer)              [(None, 100)]                   0           
________________________________________________________________________________
embedding (Embedding)               (None, 100, 256)                17152       
________________________________________________________________________________
gru (GRU)                           (None, 100, 1024)               3938304     
________________________________________________________________________________
dropout (Dropout)                   (None, 100, 1024)               0           
________________________________________________________________________________
dense (Dense)                       (None, 100, 67)                 68675       
Total params: 4,024,131
Trainable params: 4,024,131
Non-trainable params: 0
____________

In [20]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
model.fit(trainning_dataset, epochs=20)

Epoch 1/20
175/175 [==============================] - 130s 736ms/step - loss: 2.7252 - sparse_categorical_accuracy: 0.2765
Epoch 2/20
175/175 [==============================] - 127s 727ms/step - loss: 2.0035 - sparse_categorical_accuracy: 0.4140
Epoch 3/20
175/175 [==============================] - 129s 739ms/step - loss: 1.7310 - sparse_categorical_accuracy: 0.4857
Epoch 4/20
 55/175 [========>.....................] - ETA: 1:31 - loss: 1.6226 - sparse_categorical_accuracy: 0.5143

KeyboardInterrupt: 

In [21]:
inputs = keras.Input(shape=(1,), dtype=tf.int32, name="token_ids")
input_state = keras.Input(shape=(hidden_dim,), name="state")

x = layers.Embedding(vocabulary_size, embedding_dim)(inputs)
x, output_state = layers.GRU(hidden_dim, return_state=True)(x, initial_state=input_state)
outputs = layers.Dense(vocabulary_size, activation="softmax")(x)
generation_model = keras.Model(
    inputs=[inputs, input_state],
    outputs=[outputs, output_state],
)
generation_model.set_weights(model.get_weights())